In [182]:
from PyPDF2 import PdfFileReader
import os
import json
import pandas as pd

Extract information from pdf file

In [134]:
def extract_pdf(path):
    pdf=PdfFileReader(path)
    number_pages=pdf.getNumPages()
    content=''
    for i in range(number_pages):
        content+=pdf.getPage(i).extractText()
    
    content=content.replace('\\n',' ').replace('\n\n',' ')
    content=content.replace('\n',' ')
    content=content.replace('\t',' ')
    
    return content

Get path to each pdf file stored in folder  

In [135]:

def get_path_pdf(path_folder):
    path_pdf=[]
    for dirname,_,filenames in os.walk(path_folder):
        for filename in filenames:
            path_pdf.append(os.path.join(dirname,filename))
    return path_pdf


Read JD

In [136]:
path_JD="./JD/"
path_2_pdfJD=get_path_pdf(path_JD)
content_JD=extract_pdf(path_2_pdfJD[0])


Read CV

In [137]:
path_CV="./CV/"
path_2_pdfCV=get_path_pdf(path_CV)
content_CV=[]
for p in path_2_pdfCV:
    content_CV.append(extract_pdf(p))


Using Spacy to extract skill

In [158]:
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
import spacy
import re 
import pyperclip
from sympy import true

In [139]:
json_path="skill_paterns.jsonl"

with jsonlines.open(json_path) as f:
    entities=[line["label"].upper() for line in f.iter()]

# spacy model
nlp=spacy.load("en_core_web_lg")

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(json_path)

PERSONAL INFORMATION

In [181]:
mail = re.compile(r'[a-zA-Z0-9-\.]+@[a-zA-Z-\.]*\.(com|edu|net)')
git = re.compile(r'(gitlab.com/|github.com/)+[a-zA-Z0-9-\.]*')
web = re.compile(r'(http://|https:// )+[a-zA-Z0-9-"/"-\.]*')
linkedin = re.compile(r'linkedin.com/+[a-zA-Z0-9-"/"-\.]*')

def get_skill(doc):
    skill_list=[]
    for ent in doc.ents:
        if ent.label_=='SKILL':
            skill_list.append(ent.text)
    skill_list=list(set(skill_list))
    return skill_list

def get_GPA(doc):
    sent=list(doc.sents)
    gpa=''
    for s in sent:
        s=s.text
        if s.find('GPA') !=-1 :
            gba_pos= s.find('GPA') 
            for i in range (gba_pos+3,gba_pos+8):
                if (s[i] >'0' and s[i]<'9' or s[i]=='.'):
                    gpa+=s[i]
    return gpa

def get_experience(doc):
    '''
    Return\n
        None if no experience\n
        List of experience if exist
    '''
    sent=list(doc.sents)
    final_res=''
    ex=[]
    for s in sent:
        s_text=s.text.split(' ')
        flag=False
        for i in s_text:
            i=i.lower()
            if i=='experience':
                flag=True
        if flag==True:
            ex.append([s.text])
    if (len(ex)==0):
        final_res=None
    else:
        final_res=ex
    return final_res

def get_education(doc):
    sent=list(doc.sents)
    edu=''
    pos_edu=-1
    for s in sent:
        s_text=s.text.lower()
        flag=False
        if s_text.find('education')!=-1 or s_text.find('university')!=-1:
            edu =s.text
            if s_text.find('education')!=-1:
                pos_edu=s_text.find('education')
            else:
                pos_edu=s_text.find('university')
            
            edu=edu[pos_edu+10:]
    return edu

def get_phone(content_CV):
    '''
    Format for Phone-Numbers:
    134567890
    123-456-7890
    (123)(456)(7890)
    123.456.7890
    (123)(456-7890)
    '''
    phone= re.compile(r'''
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{4}|\(\d{4}\))    
    ''', re.VERBOSE) 
    matchs=phone.finditer(content_CV)
    res = ''
    for match in matchs:
        res+=match.group(0)
    if res=='':
        print(content_CV)
    return res

def get_link(link,content_CV):
    matchs=link.finditer(content_CV)
    res=''
    for match in matchs:
        res+=match.group(0)
    return res


# print(get_GPA(doc))       
# print(get_experience(doc))
# print()
# print(get_education(doc))

# print(get_link(web,test))
# print(get_link(git,test))
# print(get_link(linkedin,test))
# print(get_link(mail,test))

# get_phone(content_CV[0])

NAME

In [ ]:
from spacy.matcher import Matcher
# initialize matcher with a vocab

def extract_name(content_CV):
    matcher = Matcher(nlp.vocab)
    doc=nlp(content_CV)
    for ent in doc.ents:
        print(ent.label_,'  ',ent.text)
        if ent.label_=='PERSON':
            # print(ent.label_,'  ',ent.text)
            return(ent.text)
            
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'},{'POS': 'PROPN'},{'POS': 'PROPN'}]
    matcher.add('NAME',[pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text
extract_name(content_CV[1])

Check Accuracy of name

In [ ]:
train_data = json.load(open('./data/training/train_data.json','rb')) 

def name_acc(train_data):
    list_an=[]
    list_txt=[]
    name=[]

    for text,annotation in train_data:
        list_an.append(list(annotation.values()))
        list_txt.append(text)

    target_name=[]
    predic_name=[]

    similar=0
    for i in range(len(list_an)):
        t=''
        p=''
        p=extract_name(list_txt[i])
        predic_name.append(p)
        for j in range(len(list_an[i][0])):
            if list_an[i][0][j][2]=='Name':
                t=list_txt[i][list_an[i][0][j][0]:list_an[i][0][j][1]]
                target_name.append(t)
        if t==p:
            similar+=1
    return (similar/len(list_an)*100)    
name_acc(train_data)

In [ ]:
def extract_all_data(list_content_CV):
    

In [186]:
label=['ID','name','email','phone number','related link','Skill','experience','education']
df_information = pd.DataFrame(columns=label) 
df_information

,ID,name,email,phone number,related link,Skill,experience,education


Load accuracy name

In [ ]:
nlp_model=spacy.load('nlp_model')

Using sklearn to calculate the cosine similarity score of CV and JD

In [147]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# def cosine_similarity(list_CV,JD):
#     test=[list_CV]
test=[content_CV,content_JD]
count_matrix=cv.fit_transform(test)
cosine_similarity(count_matrix)

In [178]:
test=[content_CV[4],content_JD]
count_matrix=cv.fit_transform(test)
match=cosine_similarity(count_matrix)
match[0][1]

0.4040562886306343

In [149]:
count_matrix

<2x244 sparse matrix of type '<class 'numpy.int64'>'
	with 244 stored elements in Compressed Sparse Row format>